Here we'll pick out examples where we successfully segment images and pick out strips so we can get nice ~sxy~ examples.

In [1]:
import sys
sys.path.append('SheetIdentification')
sys.path.append('SheetIdentification/sheet_id')
sys.path.append('SheetIdentification/sheet_id/models')
sys.path.append('SheetIdentification/sheet_id/utils')
from DWD import DWD
from dwd_utils import generateFullpagePrediction, inferBoundingBoxes
from base_utils import visualizeBoundingBoxes
import numpy as np
import cv2
import matplotlib.pyplot as plt

Using TensorFlow backend.
/home/dbashir/anaconda2/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
import numpy as np
from scipy.spatial.distance import cdist
from scipy import ndimage
from scipy import signal
from scipy.stats import linregress
from scipy.signal import convolve2d, gaussian, argrelextrema
from scipy.interpolate import interp1d
from sklearn.preprocessing import normalize
from skimage.transform import rotate
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import pickle as cPickle
import librosa
import glob
import re
import sys
import os
import mido
from mido import MidiFile
import cv2
from PIL import Image

In [3]:
model = DWD(input_shape=(500,500,1), n_classes=124)
model.load_weights('dwd-finetune-100000.h5')

In [4]:
def importImage(pngfile):
    img = ndimage.imread(pngfile, flatten=True)
    img = 1 - img/255.0 # make white -> 0, black -> 1
    return img

In [5]:
def showImage(X, sz = (12,12)):
    plt.figure(figsize=sz)
    plt.imshow(1-X, cmap='gray')

In [6]:
def rotateToHorizontal(img, lb=-.5, ub=.5, incr=.02, topN=40):
    bestscore = -np.inf
    bestTheta = 0
    for theta in np.arange(lb, ub, incr):
        imgRot = rotate(img,theta)
        rsums = np.sum(imgRot, axis=1)
        rsums_sorted = sorted(rsums)[::-1]
        curscore = np.sum(rsums_sorted[0:topN])
        if curscore > bestscore:
            bestscore = curscore
            bestTheta = theta
    result = rotate(img,bestTheta)
    return result, bestTheta

In [7]:
def getBoundingBox(img, thresh=.01, buff=0.05):
    r = img.shape[0]
    c = img.shape[1]
    
    cmeans = np.mean(img, axis=0)
    cmean_thresh = np.max(cmeans) * thresh
    cidxs_above = np.argwhere(cmeans > cmean_thresh)
    col_start = cidxs_above[0][0]
    col_end = cidxs_above[-1][0]
    
    rmeans = np.mean(img, axis=1)
    rmean_thresh = np.max(rmeans) * thresh
    ridxs_above = np.argwhere(rmeans > rmean_thresh)
    row_start = ridxs_above[0][0]
    row_end = ridxs_above[-1][0]
    
    # add buffer to ignore clefs
    bufflen = np.int((col_end - col_start) * buff)
    col_start_adj = col_start + bufflen
    
    return (row_start, col_start_adj), col_end - col_start_adj, row_end - row_start

In [8]:
def showImageWithBB(X, loc, w, h, sz=(12,12)):
    fig = plt.figure(figsize=sz)
    plt.imshow(1-X, cmap='gray')
    ax = fig.gca()
    rect = patches.Rectangle((loc[1],loc[0]), w, h, linewidth=1,edgecolor='r',facecolor='none')
    ax.add_patch(rect)
    plt.show()

In [9]:
def getCropped(X, thresh=0.01):
    ll, w, h = getBoundingBox(X, thresh=thresh)
    return X[ll[0]:ll[0]+h,ll[1]:ll[1]+w]

In [10]:
def getStripBoundaries(X, showPlot = False):
    # find staves
    filt = np.vstack((np.ones(21),np.zeros(21),-1*np.ones(21))) # emphasize horizontal lines
    Xfilt = signal.convolve2d(X-np.mean(X), filt, mode='same')
    rsum = np.sum(Xfilt, axis=1)
    rsum_spectra = librosa.core.stft(rsum,n_fft=32,hop_length=1) # look for high frequency oscillations
    spectra_max = np.max(rsum_spectra[4:10,:], axis=0)
    spectra_max_smoothed = np.convolve(spectra_max,np.hanning(15),'same')
    peakidxs = signal.find_peaks_cwt(spectra_max_smoothed, range(15,41,1))
    #assert(len(peakidxs) % 2 == 0)
    
    # determine boundaries
    numStrips = np.int(len(peakidxs))
    boundaries = []
    for i in range(numStrips):
        peak1idx = peakidxs[i]
        if i >= (numStrips - 1):
            break
        peak2idx = peakidxs[i+1]
        midpt = np.int((peak1idx + peak2idx)/2.0)
        boundaries.append(midpt)
    boundaries.insert(0, 0)  # for first strip
    boundaries.append(X.shape[0]-1) # for last strip
    
    # show peak picking results (for debugging)
    if showPlot:
        plt.plot(spectra_max_smoothed)
        for pk in peakidxs:
            plt.axvline(x=pk, color='r', linewidth=1)
            
    return boundaries

In [11]:
def visualizeBoundaries(X, bnds, savefile = None):
    showImage(X)
    for bnd in bnds:
        plt.axhline(y=bnd, color='r', linewidth=1)
    if savefile:
        plt.savefig(savefile)

In [12]:
def getStrips(X, bnds):
    strips = []
    for i in range(len(bnds)-1):
        rstart = bnds[i]
        rend = bnds[i+1] + 1 # include endpoint
        chunk = X[rstart:rend,:]
        strips.append(chunk)
    return strips

In [13]:
def showStrips(strips):
    for strip in strips:
        showImage(strip)

In [14]:
def locateStaffLines(s, delta = 3, debug=False):
    medvals = np.median(s, axis=1)
    if debug:
        print("num medvals: " + str(len(medvals)))
        print("medvals are: " +  str(medvals))
    for i, curVal in enumerate(medvals):
        if i > delta and i < len(medvals)-delta:
            if curVal != np.max(medvals[i-delta:i+delta+1]): # only keep local maxima
                medvals[i] = 0
    idxs_sorted_val = np.argsort(medvals)[::-1]
    idxs_lines = sorted(idxs_sorted_val[0:10]) # going to change from [0:10] to [0:10] skipping by 2
    idxs_lines = idxs_lines[1::2] # we want just 5, so start at the second and skip every other.
    if debug:
        print("sorted idxs val before clip: " + str(idxs_sorted_val))
        print("num sorted idxs: " + str(len(idxs_sorted_val)))
        print("idxs_lines number after clip: " + str(len(idxs_lines)))
        print("idxs_lines: " + str(idxs_lines))
    return np.array(idxs_lines)

In [15]:
def estimateMidiNum2PixelRowMapping(s, lineIdxs):
    x = np.array([4, 2, 0, -2, -4]) # midi nums for staff lines, reformatted to be 0 in the middle, etc.
    m, b, _, _, _ = linregress(x, lineIdxs)
    return (m,b)

In [16]:
def estimatePixelRow (midinum, m, b):
    return np.int(np.round(midinum * m + b))

In [17]:
def estimatePixelRow2MidiNumMapping(s, lineIdxs):
    x = np.array([4, 2, 0, -2, -4]) # midi nums for staff lines
    m, b, _, _, _ = linregress(lineIdxs, x)
    return (m,b)

In [18]:
def getStripBoundariesNonPiano(X, showPlot = False):
    # find staves
    filt = np.vstack((np.ones(21),np.zeros(21),-1*np.ones(21))) # emphasize horizontal lines
    Xfilt = signal.convolve2d(X-np.mean(X), filt, mode='same')
    rsum = np.sum(Xfilt, axis=1)
    rsum_spectra = librosa.core.stft(rsum,n_fft=32,hop_length=1) # look for high frequency oscillations
    spectra_max = np.max(rsum_spectra[4:10,:], axis=0)
    spectra_max_smoothed = np.convolve(spectra_max,np.hanning(15),'same')
    peakidxs = signal.find_peaks_cwt(spectra_max_smoothed, range(15,41,1))
    #assert(len(peakidxs) % 2 == 0)
    
    # determine boundaries
    numStrips = np.int(len(peakidxs)/2) # note the lack of -1 here
    boundaries = []
    for i in range(numStrips):
        peak1idx = peakidxs[2*i]
        peak2idx = peakidxs[2*i+1]
        midpt = np.int((peak1idx + peak2idx)/2.0)
        boundaries.append(midpt)
    boundaries.insert(0, 0)  # for first strip
    boundaries.append(X.shape[0]-1) # for last strip
    
    # show peak picking results (for debugging)
    if showPlot:
        plt.plot(spectra_max_smoothed)
        for pk in peakidxs:
            plt.axvline(x=pk, color='r', linewidth=1)
            
    return boundaries

In [19]:
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle
def visualizeBoundingBoxes(img, boxes, figsize=(20, 20)):
    """
    Visualize bounding boxes on the provided image
    """
    _, ax = plt.subplots(1, 1, figsize=figsize)
    ax.imshow(img, cmap='gray')
    patches = []
    for box in boxes:
        patches.append(Rectangle((box[0], box[1]), box[2]-box[0], box[3]-box[1]))
    pc = PatchCollection(patches, facecolor='None', edgecolor='r')
    ax.add_collection(pc)

In [22]:
def getStrips(image, thresh, debug=False):
    I = importImage(image)
    Irot, theta = rotateToHorizontal(I)
    Icrop = getCropped(Irot, thresh=thresh)
    bnds = getStripBoundariesNonPiano(Icrop)
    if debug:
        ll, w, h = getBoundingBox(Irot, thresh=thresh)
        showImageWithBB(Irot, ll, bbw, bbh)
        visualizeBoundaries(Icrop, bnds)
    strips = getStrips(Icrop, bnds)
    #TODO: figure out how to find first strip
    return strips

In [21]:
def get_lines_bboxes(strip, debug=False):
    lines = locateStaffLines(strip)
    inv_s = (255*(1-strip)).astype(int)
    energy_map, class_map, bbox_map = generateFullpagePrediction(model, inv_s)
    boxes, scores = inferBoundingBoxes(energy_map, class_map, bbox_map, non_max_suppression_factor=0.25)
    if debug:
        print("boxes: " + str(boxes))
        print("energy map: " + str(energy_map))
    return (inv_s,boxes,lines)
    # you'd do smth like (inv_s, boxes, lines = get_lines_bboxes)

In [23]:
def visualize_output(s, inv_s, lines, boxes):
    visualizeBoundingBoxes(inv_s, boxes)
    for line in lines:
        plt.axhline(y=line, color='r', linewidth=1)
    plt.show()
    
    value_array = []
    y_vals = []
    notes = []
    for box in boxes:
        x1 = box[0]
        x2 = box[2]
        y1 = box[1]
        y2 = box[3]
        x_val = (x1+x2)/2
        y_val = (y1+y2)/2
        value_array.append((x_val, y_val))
        y_vals.append(y_val)
    for y_val in y_vals:
        m, b = estimatePixelRow2MidiNumMapping(s, lines[0:5])
        estNote = estimatePixelRow(y_val, m, b)
        notes.append(estNote)
    print('notes at location x: ')
    print(notes)

In [24]:
pics = os.listdir('pdf/mini_dataset_png')

In [25]:
pic_names = []
for pic in pics:
    name = os.path.join('pdf/mini_dataset_png', pic)
    pic_names.append(name)

In [ ]:
pic = pic_names[0]
I = importImage(pic)
Irot, theta = rotateToHorizontal(I)
Icrop = getCropped(Irot, thresh=0.2) # with default thresh
bnds = getStripBoundariesNonPiano(Icrop)
bnds_piano = getStripBoundaries(Icrop, debug=True)

/home/dbashir/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:2: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0.
Use ``matplotlib.pyplot.imread`` instead.
  from ipykernel import kernelapp as app


In [ ]:
visualizeBoundaries(Icrop, bnds)